In [1]:
import sys
print(sys.executable)




C:\Users\Johanna\anaconda3\envs\AI_project\python.exe


In [40]:
!pip install scikeras

In [144]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor
from tensorflow.keras.layers import  Dropout
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from tensorflow.keras.layers import Input
import joblib
import matplotlib.pyplot as plt
from datetime import time


In [134]:
#df_lagged_all = pd.read_parquet("data/processed/df_lagged_30_all.parquet")
df_lagged_all = pd.read_parquet("data/processed/df_lagged_all.parquet")
df_lagged_all_peak = pd.read_parquet("data/processed/df_lagged_all_peak.parquet")

df_2_full = pd.read_parquet("data/processed/df_2_full_v2.parquet")

df_all=pd.read_parquet("data/processed/df_all.parquet")
df_all_peak= df_all.set_index("Datetime").between_time("07:15", "08:30").reset_index()



In [5]:
target_sensor = 1076
same_portal="55620"
neighbour_portal = "56160"

same_portal_sensors = df_2_full[df_2_full['PORTAL_clean'] == same_portal]['DP_ID'].unique()
same_sensors = [s for s in same_portal_sensors if s != target_sensor]


neighbour_sensors = df_2_full[df_2_full['PORTAL_clean'] == neighbour_portal]['DP_ID'].unique()
all_sensors=df_2_full['DP_ID'].unique()
except_target_sensors = [s for s in all_sensors if s != target_sensor]




In [69]:
nlags=15
#nlags=30

In [218]:


flow_features_same = [
    f'SENSOR_{sensor}_FLOW_lag_{i+1}'
    for sensor in same_sensors
    for i in range(nlags)
]
flow_features_neighbour = [
    f'SENSOR_{sensor}_FLOW_lag_{i+1}'
    for sensor in neighbour_sensors
    for i in range(nlags)
]
speed_features_same = [
    f'SENSOR_{sensor}_SPEED_lag_{i+1}'
    for sensor in same_sensors
    for i in range(nlags)
]
speed_features_neighbour = [
    f'SENSOR_{sensor}_SPEED_lag_{i+1}'
    for sensor in neighbour_sensors
    for i in range(nlags)
]        

xgb-boost : hyperparameter tuning

In [135]:
df_lagged_all_peak.head()

,Datetime,SENSOR_539_FLOW_lag_1,SENSOR_539_FLOW_lag_2,SENSOR_539_FLOW_lag_3,SENSOR_539_FLOW_lag_4,SENSOR_539_FLOW_lag_5,SENSOR_539_FLOW_lag_6,SENSOR_539_FLOW_lag_7,SENSOR_539_FLOW_lag_8,SENSOR_539_FLOW_lag_9,...,SENSOR_1254_SPEED_lag_10,SENSOR_1254_SPEED_lag_11,SENSOR_1254_SPEED_lag_12,SENSOR_1254_SPEED_lag_13,SENSOR_1254_SPEED_lag_14,SENSOR_1254_SPEED_lag_15,FLOW_filled,SPEED_MS_AVG_filled,FLOW_future_sum,SPEED_future_mean
0,2021-06-01 07:30:00,17.0,11.0,18.0,13.0,20.0,13.0,18.0,10.0,17.0,...,22.30,22.35,22.30,23.17,23.56,24.36,16.0,18.95,304.0,18.292000
1,2021-06-01 07:31:00,12.0,17.0,11.0,18.0,13.0,20.0,13.0,18.0,10.0,...,22.27,22.30,22.35,22.30,23.17,23.56,20.0,18.36,311.0,18.148667
2,2021-06-01 07:32:00,17.0,12.0,17.0,11.0,18.0,13.0,20.0,13.0,18.0,...,23.91,22.27,22.30,22.35,22.30,23.17,19.0,18.53,309.0,18.065333
3,2021-06-01 07:33:00,12.0,17.0,12.0,17.0,11.0,18.0,13.0,20.0,13.0,...,22.80,23.91,22.27,22.30,22.35,22.30,19.0,19.97,310.0,17.988667
4,2021-06-01 07:34:00,15.0,12.0,17.0,12.0,17.0,11.0,18.0,13.0,20.0,...,22.79,22.80,23.91,22.27,22.30,22.35,17.0,19.69,307.0,17.882000


In [148]:
#df_train, df_test = train_test_split(df_lagged_all_peak, test_size=0.2, random_state=42)
df_train, df_test = train_test_split(df_lagged_all, test_size=0.2, random_state=42)

#split_time = df_2_full['Datetime'].quantile(0.8)
#print(split_time)

#df_train = df_lagged_all[df_lagged_all['Datetime'] <= split_time]
#df_test = df_lagged_all[df_lagged_all['Datetime'] > split_time]


In [137]:
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.001,0.005,0.01, 0.05],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0]
}


In [138]:


random_search_flow = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30,  # Anzahl zufälliger Kombinationen
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)

random_search_flow.fit(df_train[flow_features_same], df_train["FLOW_future_sum"])
print("Beste Parameter (FLOW):", random_search_flow.best_params_)
model_flow = random_search_flow.best_estimator_
model_flow.save_model("model/xgb_flow_same.json")

y_pred = model_flow.predict(df_test[flow_features_same])
y_test = df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


random_search_speed = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30,  # Anzahl zufälliger Kombinationen
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)


random_search_speed.fit(df_train[speed_features_same], df_train["SPEED_future_mean"])
print("Beste Parameter (FLOW):", random_search_speed.best_params_)
model_speed = random_search_speed.best_estimator_

y_pred = model_speed.predict(df_test[speed_features_same])
y_test = df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Beste Parameter (FLOW): {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
XGBoost Regression (FLOW) -> RMSE: 24.194, MAE: 18.232, R²: 0.834
Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Beste Parameter (FLOW): {'subsample': 0.6, 'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
XGBoost Regression (SPEED) -> RMSE: 0.564, MAE: 0.329, R²: 0.689


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [139]:
#neighbour portal

random_search_flow = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30,  # Anzahl zufälliger Kombinationen
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)

random_search_flow.fit(df_train[flow_features_neighbour], df_train["FLOW_future_sum"])
print("Beste Parameter (FLOW):", random_search_flow.best_params_)
model_flow = random_search_flow.best_estimator_
y_pred = model_flow.predict(df_test[flow_features_neighbour])
y_test = df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


random_search_speed = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30,  # Anzahl zufälliger Kombinationen
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)


random_search_speed.fit(df_train[speed_features_neighbour], df_train["SPEED_future_mean"])
print("Beste Parameter (FLOW):", random_search_speed.best_params_)
model_speed = random_search_speed.best_estimator_

y_pred = model_speed.predict(df_test[speed_features_neighbour])
y_test = df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Beste Parameter (FLOW): {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
XGBoost Regression (FLOW) -> RMSE: 20.202, MAE: 15.353, R²: 0.884
Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Beste Parameter (FLOW): {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
XGBoost Regression (SPEED) -> RMSE: 0.632, MAE: 0.332, R²: 0.610


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Feedforward Neural Network (NN)

In [140]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[flow_features_same])
X_test_scaled  = scaler.transform(df_test[flow_features_same])


model_flow= Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Eingabeschicht
    Dense(32, activation='relu'),                                           # Versteckte Schicht
    Dense(16, activation='relu'),                                           # Weitere versteckte Schicht
    Dense(1)                                                                 # Ausgangsschicht für Regression
])

model_flow.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_flow = model_flow.fit(
    X_train_scaled, df_train["FLOW_future_sum"],
    validation_split=0.2,  # 20% der Trainingsdaten für Validierung
    epochs=50,
    batch_size=32,
    verbose=0
)



y_pred = model_flow.predict(X_test_scaled)
y_test=df_test["FLOW_future_sum"]

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[speed_features_same])
X_test_scaled  = scaler.transform(df_test[speed_features_same])

model_speed = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Eingabeschicht
    Dense(32, activation='relu'),                                           # Versteckte Schicht
    Dense(16, activation='relu'),                                           # Weitere versteckte Schicht
    Dense(1)                                                                 # Ausgangsschicht für Regression
])

model_speed.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_speed = model_speed.fit(
    X_train_scaled, df_train["SPEED_future_mean"],
    validation_split=0.2,  # 20% der Trainingsdaten für Validierung
    epochs=50,
    batch_size=32,
    verbose=0
)


y_pred = model_speed.predict(X_test_scaled)
y_test=df_test["SPEED_future_mean"]
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")



C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fast forward NN (FLOW) -> RMSE: 25.268, MAE: 19.388, R²: 0.819


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Fast forward NN (SPEED) -> RMSE: 0.642, MAE: 0.355, R²: 0.597


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [141]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[flow_features_neighbour])
X_test_scaled  = scaler.transform(df_test[flow_features_neighbour])


model_flow= Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Eingabeschicht
    Dense(32, activation='relu'),                                           # Versteckte Schicht
    Dense(16, activation='relu'),                                           # Weitere versteckte Schicht
    Dense(1)                                                                 # Ausgangsschicht für Regression
])

model_flow.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_flow = model_flow.fit(
    X_train_scaled, df_train["FLOW_future_sum"],
    validation_split=0.2,  # 20% der Trainingsdaten für Validierung
    epochs=50,
    batch_size=32,
    verbose=0
)



y_pred = model_flow.predict(X_test_scaled)
y_test=df_test["FLOW_future_sum"]

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[speed_features_neighbour])
X_test_scaled  = scaler.transform(df_test[speed_features_neighbour])

model_speed = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Eingabeschicht
    Dense(32, activation='relu'),                                           # Versteckte Schicht
    Dense(16, activation='relu'),                                           # Weitere versteckte Schicht
    Dense(1)                                                                 # Ausgangsschicht für Regression
])

model_speed.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_speed = model_speed.fit(
    X_train_scaled, df_train["SPEED_future_mean"],
    validation_split=0.2,  # 20% der Trainingsdaten für Validierung
    epochs=50,
    batch_size=32,
    verbose=0
)


y_pred = model_speed.predict(X_test_scaled)
y_test=df_test["SPEED_future_mean"]
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")



C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fast forward NN (FLOW) -> RMSE: 22.509, MAE: 17.109, R²: 0.856


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fast forward NN (SPEED) -> RMSE: 0.695, MAE: 0.375, R²: 0.529


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [152]:
scaler_flow_same = StandardScaler()
X_train_scaled = scaler_flow_same.fit_transform(df_train[flow_features_same])
X_test_scaled  = scaler_flow_same.transform(df_test[flow_features_same])
joblib.dump(scaler_flow_same, "model/scaler_flow_same.pkl")

['model/scaler_flow_same.pkl']

In [87]:
#same portal-flow
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[flow_features_same])
X_test_scaled  = scaler.transform(df_test[flow_features_same])
y_train = df_train["FLOW_future_sum"]
y_test = df_test["FLOW_future_sum"]

number_neurons = [16, 32, 64, 128]
number_layers = [2, 3,4]
dropout_rates = [0.0, 0.05, 0.1]

grid_search_df = pd.DataFrame(columns=["number_neurons","number_layers","dropout_rate","train_rmse", "val_rmse"])

for neurons in number_neurons:
    for num_layer in number_layers:
        for dropout_rate in dropout_rates:
            model = Sequential()
            for _ in range(num_layer):
                model.add(Dense(neurons, activation='relu'))
                if dropout_rate > 0:
                    model.add(Dropout(dropout_rate))
            model.add(Dense(1))
            optimizer = Adam(learning_rate=0.01)
            model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
            early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
    
            hist = model.fit(
                X_train_scaled, y_train,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr],
                verbose=0
            )
    
            # Ergebnisse speichern
            min_err = min(hist.history['rmse'])
            min_val_err = min(hist.history['val_rmse'])
            it_df = pd.DataFrame({
                "number_neurons": [neurons],
                "number_layers": [num_layer],
                "dropout_rate":[dropout_rate],
                "train_rmse": [min_err],
                "val_rmse": [min_val_err]
            })
            grid_search_df = pd.concat([grid_search_df, it_df], axis=0)
            print(neurons, num_layer,dropout_rate, "done")

# Übersicht
grid_search_df.reset_index(drop=True, inplace=True)
print(grid_search_df)
best_config = grid_search_df.loc[grid_search_df['val_rmse'].idxmin()]
best_neurons = int(best_config['number_neurons'])
best_layers = int(best_config['number_layers'])
best_dropout = int(best_config['dropout_rate'])

# Modell mit besten Parametern
best_model = Sequential()
for _ in range(best_layers):
    best_model.add(Dense(best_neurons, activation='relu'))
    if best_dropout > 0:
        best_model.add(Dropout(best_dropout))
best_model.add(Dense(1))

optimizer = Adam(learning_rate=0.01)
best_model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("model/NN_model_flow_same_best.keras", monitor='val_rmse', save_best_only=True, mode='min')

# Training
best_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=0
)

# Testbewertung
y_pred = best_model.predict(X_test_scaled)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Best FLOW NN -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")






C:\Users\Johanna\AppData\Local\Temp\ipykernel_10492\308595228.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grid_search_df = pd.concat([grid_search_df, it_df], axis=0)


16 2 0.0 done
16 2 0.05 done
16 2 0.1 done
16 3 0.0 done
16 3 0.05 done
16 3 0.1 done
16 4 0.0 done
16 4 0.05 done
16 4 0.1 done
32 2 0.0 done
32 2 0.05 done
32 2 0.1 done
32 3 0.0 done
32 3 0.05 done
32 3 0.1 done
32 4 0.0 done
32 4 0.05 done
32 4 0.1 done
64 2 0.0 done
64 2 0.05 done
64 2 0.1 done
64 3 0.0 done
64 3 0.05 done
64 3 0.1 done
64 4 0.0 done
64 4 0.05 done
64 4 0.1 done
128 2 0.0 done
128 2 0.05 done
128 2 0.1 done
128 3 0.0 done
128 3 0.05 done
128 3 0.1 done
128 4 0.0 done
128 4 0.05 done
128 4 0.1 done
   number_neurons number_layers  dropout_rate  train_rmse   val_rmse
0              16             2          0.00   29.371403  29.733335
1              16             2          0.05   37.075043  33.153324
2              16             2          0.10   42.041279  33.635246
3              16             3          0.00   27.912714  28.976200
4              16             3          0.05   39.897415  33.503639
5              16             3          0.10   43.846443  34

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [151]:
scaler_speed_same = StandardScaler()
X_train_scaled = scaler_speed_same.fit_transform(df_train[speed_features_same])
X_test_scaled  = scaler_speed_same.transform(df_test[speed_features_same])
joblib.dump(scaler_speed_same, "model/scaler_speed_same.pkl")

['model/scaler_speed_same.pkl']

In [89]:
#same portal-speed
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[speed_features_same])
X_test_scaled  = scaler.transform(df_test[speed_features_same])
y_train = df_train["SPEED_future_mean"]
y_test = df_test["SPEED_future_mean"]

number_neurons = [16, 32, 64, 128]
number_layers = [2, 3,4]
dropout_rates = [0.0, 0.05, 0.1]

grid_search_df = pd.DataFrame(columns=["number_neurons","number_layers","dropout_rate","train_rmse", "val_rmse"])

for neurons in number_neurons:
    for num_layer in number_layers:
        for dropout_rate in dropout_rates:
            model = Sequential()
            for _ in range(num_layer):
                model.add(Dense(neurons, activation='relu'))
                if dropout_rate > 0:
                    model.add(Dropout(dropout_rate))
            model.add(Dense(1))
            optimizer = Adam(learning_rate=0.01)
            model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
            early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
    
            # Optional: speichern der besten Gewichte
            #checkpoint = ModelCheckpoint("model/NN_model_speed_same_best.keras", monitor='val_rmse', save_best_only=True, mode='min')
    
            hist = model.fit(
                X_train_scaled, y_train,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr],
                verbose=0
            )
    
            # Ergebnisse speichern
            min_err = min(hist.history['rmse'])
            min_val_err = min(hist.history['val_rmse'])
            it_df = pd.DataFrame({
                "number_neurons": [neurons],
                "number_layers": [num_layer],
                "dropout_rate":[dropout_rate],
                "train_rmse": [min_err],
                "val_rmse": [min_val_err]
            })
            grid_search_df = pd.concat([grid_search_df, it_df], axis=0)
            print(neurons, num_layer,dropout_rate, "done")

# Übersicht
grid_search_df.reset_index(drop=True, inplace=True)
print(grid_search_df)
best_config = grid_search_df.loc[grid_search_df['val_rmse'].idxmin()]
best_neurons = int(best_config['number_neurons'])
best_layers = int(best_config['number_layers'])
best_dropout = int(best_config['dropout_rate'])

# Modell mit besten Parametern
best_model = Sequential()
for _ in range(best_layers):
    best_model.add(Dense(best_neurons, activation='relu'))
    if best_dropout > 0:
        best_model.add(Dropout(best_dropout))
best_model.add(Dense(1))

optimizer = Adam(learning_rate=0.01)
best_model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("model/NN_model_speed_same_best.keras", monitor='val_rmse', save_best_only=True, mode='min')

# Training
best_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=0
)

# Testbewertung
y_pred = best_model.predict(X_test_scaled)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Best SPEED NN -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")






C:\Users\Johanna\AppData\Local\Temp\ipykernel_10492\867997906.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grid_search_df = pd.concat([grid_search_df, it_df], axis=0)


16 2 0.0 done
16 2 0.05 done
16 2 0.1 done
16 3 0.0 done
16 3 0.05 done
16 3 0.1 done
16 4 0.0 done
16 4 0.05 done
16 4 0.1 done
32 2 0.0 done
32 2 0.05 done
32 2 0.1 done
32 3 0.0 done
32 3 0.05 done
32 3 0.1 done
32 4 0.0 done
32 4 0.05 done
32 4 0.1 done
64 2 0.0 done
64 2 0.05 done
64 2 0.1 done
64 3 0.0 done
64 3 0.05 done
64 3 0.1 done
64 4 0.0 done
64 4 0.05 done
64 4 0.1 done
128 2 0.0 done
128 2 0.05 done
128 2 0.1 done
128 3 0.0 done
128 3 0.05 done
128 3 0.1 done
128 4 0.0 done
128 4 0.05 done
128 4 0.1 done
   number_neurons number_layers  dropout_rate  train_rmse  val_rmse
0              16             2          0.00    0.805148  0.790606
1              16             2          0.05    0.973746  0.811842
2              16             2          0.10    0.822249  0.793180
3              16             3          0.00    0.733872  0.737610
4              16             3          0.05    0.812134  0.754808
5              16             3          0.10    0.878491  0.784833

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [91]:
#neighbour portal-flow
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[flow_features_neighbour])
X_test_scaled  = scaler.transform(df_test[flow_features_neighbour])
y_train = df_train["FLOW_future_sum"]
y_test = df_test["FLOW_future_sum"]

number_neurons = [16, 32, 64, 128]
number_layers = [2, 3,4]
dropout_rates = [0.0, 0.05, 0.1]

grid_search_df = pd.DataFrame(columns=["number_neurons","number_layers","dropout_rate","train_rmse", "val_rmse"])

for neurons in number_neurons:
    for num_layer in number_layers:
        for dropout_rate in dropout_rates:
            model = Sequential()
            for _ in range(num_layer):
                model.add(Dense(neurons, activation='relu'))
                if dropout_rate > 0:
                    model.add(Dropout(dropout_rate))
            model.add(Dense(1))
            optimizer = Adam(learning_rate=0.01)
            model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
            early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
    
            # Optional: speichern der besten Gewichte
            #checkpoint = ModelCheckpoint("model/NN_model_flow_neighbour_best.keras", monitor='val_rmse', save_best_only=True, mode='min')
    
            hist = model.fit(
                X_train_scaled, y_train,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr],
                verbose=0
            )
    
            # Ergebnisse speichern
            min_err = min(hist.history['rmse'])
            min_val_err = min(hist.history['val_rmse'])
            it_df = pd.DataFrame({
                "number_neurons": [neurons],
                "number_layers": [num_layer],
                "dropout_rate":[dropout_rate],
                "train_rmse": [min_err],
                "val_rmse": [min_val_err]
            })
            grid_search_df = pd.concat([grid_search_df, it_df], axis=0)
            print(neurons, num_layer,dropout_rate, "done")

# Übersicht
grid_search_df.reset_index(drop=True, inplace=True)
print(grid_search_df)
best_config = grid_search_df.loc[grid_search_df['val_rmse'].idxmin()]
best_neurons = int(best_config['number_neurons'])
best_layers = int(best_config['number_layers'])
best_dropout = int(best_config['dropout_rate'])
print(best_neurons, best_layers, best_dropout)

# Modell mit besten Parametern
best_model = Sequential()
for _ in range(best_layers):
    best_model.add(Dense(best_neurons, activation='relu'))
    if best_dropout > 0:
        best_model.add(Dropout(best_dropout))
best_model.add(Dense(1))

optimizer = Adam(learning_rate=0.01)
best_model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("model/NN_model_flow_neighbour_best.keras", monitor='val_rmse', save_best_only=True, mode='min')

# Training
best_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=0
)

# Testbewertung
y_pred = best_model.predict(X_test_scaled)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Best FLOW NN -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")






C:\Users\Johanna\AppData\Local\Temp\ipykernel_10492\2257366272.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grid_search_df = pd.concat([grid_search_df, it_df], axis=0)


16 2 0.0 done
16 2 0.05 done
16 2 0.1 done
16 3 0.0 done
16 3 0.05 done
16 3 0.1 done
16 4 0.0 done
16 4 0.05 done
16 4 0.1 done
32 2 0.0 done
32 2 0.05 done
32 2 0.1 done
32 3 0.0 done
32 3 0.05 done
32 3 0.1 done
32 4 0.0 done
32 4 0.05 done
32 4 0.1 done
64 2 0.0 done
64 2 0.05 done
64 2 0.1 done
64 3 0.0 done
64 3 0.05 done
64 3 0.1 done
64 4 0.0 done
64 4 0.05 done
64 4 0.1 done
128 2 0.0 done
128 2 0.05 done
128 2 0.1 done
128 3 0.0 done
128 3 0.05 done
128 3 0.1 done
128 4 0.0 done
128 4 0.05 done
128 4 0.1 done
   number_neurons number_layers  dropout_rate  train_rmse   val_rmse
0              16             2          0.00   24.768108  25.254890
1              16             2          0.05   29.870770  26.075983
2              16             2          0.10   33.876259  25.873125
3              16             3          0.00   23.947803  24.581074
4              16             3          0.05   32.065853  28.567739
5              16             3          0.10   34.205040  26

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [93]:
#neighbour portal-speed
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[speed_features_neighbour])
X_test_scaled  = scaler.transform(df_test[speed_features_neighbour])
y_train = df_train["SPEED_future_mean"]
y_test = df_test["SPEED_future_mean"]

number_neurons = [16, 32, 64, 128]
number_layers = [2, 3,4]
dropout_rates = [0.0, 0.05, 0.1]

grid_search_df = pd.DataFrame(columns=["number_neurons","number_layers","dropout_rate","train_rmse", "val_rmse"])

for neurons in number_neurons:
    for num_layer in number_layers:
        for dropout_rate in dropout_rates:
            model = Sequential()
            for _ in range(num_layer):
                model.add(Dense(neurons, activation='relu'))
                if dropout_rate > 0:
                    model.add(Dropout(dropout_rate))
            model.add(Dense(1))
            optimizer = Adam(learning_rate=0.01)
            model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
            early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
    
            # Optional: speichern der besten Gewichte
            #checkpoint = ModelCheckpoint("model/NN_model_speed_neighbour_best.keras", monitor='val_rmse', save_best_only=True, mode='min')
    
            hist = model.fit(
                X_train_scaled, y_train,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr],
                verbose=0
            )
    
            # Ergebnisse speichern
            min_err = min(hist.history['rmse'])
            min_val_err = min(hist.history['val_rmse'])
            it_df = pd.DataFrame({
                "number_neurons": [neurons],
                "number_layers": [num_layer],
                "dropout_rate":[dropout_rate],
                "train_rmse": [min_err],
                "val_rmse": [min_val_err]
            })
            grid_search_df = pd.concat([grid_search_df, it_df], axis=0)
            print(neurons, num_layer,dropout_rate, "done")

# Übersicht
grid_search_df.reset_index(drop=True, inplace=True)
print(grid_search_df)
best_config = grid_search_df.loc[grid_search_df['val_rmse'].idxmin()]
best_neurons = int(best_config['number_neurons'])
best_layers = int(best_config['number_layers'])
best_dropout = int(best_config['dropout_rate'])
print(best_neurons, best_layers, best_dropout)

# Modell mit besten Parametern
best_model = Sequential()
for _ in range(best_layers):
    best_model.add(Dense(best_neurons, activation='relu'))
    if best_dropout > 0:
        best_model.add(Dropout(best_dropout))
best_model.add(Dense(1))

optimizer = Adam(learning_rate=0.01)
best_model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("model/NN_model_speed_neighbour_best.keras", monitor='val_rmse', save_best_only=True, mode='min')

# Training
best_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=0
)

# Testbewertung
y_pred = best_model.predict(X_test_scaled)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Best SPEED NN -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")






C:\Users\Johanna\AppData\Local\Temp\ipykernel_10492\326426236.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grid_search_df = pd.concat([grid_search_df, it_df], axis=0)


16 2 0.0 done
16 2 0.05 done
16 2 0.1 done
16 3 0.0 done
16 3 0.05 done
16 3 0.1 done
16 4 0.0 done
16 4 0.05 done
16 4 0.1 done
32 2 0.0 done
32 2 0.05 done
32 2 0.1 done
32 3 0.0 done
32 3 0.05 done
32 3 0.1 done
32 4 0.0 done
32 4 0.05 done
32 4 0.1 done
64 2 0.0 done
64 2 0.05 done
64 2 0.1 done
64 3 0.0 done
64 3 0.05 done
64 3 0.1 done
64 4 0.0 done
64 4 0.05 done
64 4 0.1 done
128 2 0.0 done
128 2 0.05 done
128 2 0.1 done
128 3 0.0 done
128 3 0.05 done
128 3 0.1 done
128 4 0.0 done
128 4 0.05 done
128 4 0.1 done
   number_neurons number_layers  dropout_rate  train_rmse  val_rmse
0              16             2          0.00    0.917047  0.926511
1              16             2          0.05    0.908428  0.900218
2              16             2          0.10    0.936143  0.904106
3              16             3          0.00    0.855502  0.891089
4              16             3          0.05    0.913606  0.893798
5              16             3          0.10    0.961896  0.917284

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


LSTM

In [155]:


flow_features_same = [
    f'SENSOR_{sensor}_FLOW'
    for sensor in same_sensors
    #for i in range(nlags)
]
flow_features_neighbour = [
    f'SENSOR_{sensor}_FLOW'
    for sensor in neighbour_sensors
    #for i in range(nlags)
]
speed_features_same = [
    f'SENSOR_{sensor}_SPEED'
    for sensor in same_sensors
    #for i in range(nlags)
]
speed_features_neighbour = [
    f'SENSOR_{sensor}_SPEED'
    for sensor in neighbour_sensors
    #for i in range(nlags)
]        

In [100]:
def create_sequences(X, y, seq_length=15, horizon=15):
    Xs, ys = [], []
    for i in range(len(X) - seq_length - horizon + 1):
        Xs.append(X[i:i+seq_length])
        ys.append(y[i+seq_length:i+seq_length+horizon])
    return np.array(Xs), np.array(ys)


In [216]:

X = df_all[flow_features_neighbour].values
y = df_all[f'SENSOR_{target_sensor}_FLOW'].values



X_seq, y_seq = create_sequences(X, y, seq_length=15, horizon=15)

X_scaler = StandardScaler()
y_scaler = StandardScaler()

# reshape für den Scaler (2D erwartet)
X_scaled = X_scaler.fit_transform(X_seq.reshape(-1, X_seq.shape[2])).reshape(X_seq.shape)
#y_scaled = y_scaler.fit_transform(y_seq.reshape(-1, 1))
X_seq=X_scaled
#y_seq=y_scaled


n_features = X_seq.shape[2]

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15,n_features), return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=False),
    Dense(15)  # 15 Output-Werte für 15-Minuten-Vorhersage
])

model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

model.fit(X_train, y_train, epochs=20, batch_size=32,callbacks=[early_stop, reduce_lr, checkpoint], validation_split=0.1,verbose=1)


y_pred = model.predict(X_test)





Epoch 1/20


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1642/1642 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 31.4992 - rmse: 5.6124 - val_loss: 19.0745 - val_rmse: 4.3674 - learning_rate: 0.0010
Epoch 2/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - loss: 16.3062 - rmse: 4.0381 - val_loss: 18.9920 - val_rmse: 4.3580 - learning_rate: 0.0010
Epoch 3/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - loss: 16.2274 - rmse: 4.0283 - val_loss: 18.7797 - val_rmse: 4.3336 - learning_rate: 0.0010
Epoch 4/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - loss: 16.1265 - rmse: 4.0158 - val_loss: 18.7793 - val_rmse: 4.3335 - learning_rate: 5.0000e-04
Epoch 5/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - loss: 16.0776 - rmse: 4.0097 - val_loss: 18.7620 - val_rmse: 4.3315 - learning_rate: 5.0000e-04
457/457 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


In [217]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"LSTM (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

y_test_sum = y_test.sum(axis=1)
y_pred_sum = y_pred.sum(axis=1)

# Metriken berechnen
rmse = mean_squared_error(y_test_sum, y_pred_sum, squared=False)
mae = mean_absolute_error(y_test_sum, y_pred_sum)
r2 = r2_score(y_test_sum, y_pred_sum)

print(f"LSTM Sum Forecast (FLOW) → RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

LSTM (FLOW) -> RMSE: 3.936, MAE: 2.919, R2: 0.643
LSTM Sum Forecast (FLOW) → RMSE: 35.601, MAE: 22.823, R²: 0.833


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [158]:
df_all['FLOW_future_sum'] = (
    df_all[f'SENSOR_{target_sensor}_FLOW']
    .rolling(15, min_periods=15)
    .sum()
    .shift(-14)  # damit die Summe bei t=07:02 die Werte von 07:02–07:16 enthält
    .reset_index(level=0, drop=True)
)
df_all['SPEED_future_mean'] = (
    df_all[f'SENSOR_{target_sensor}_SPEED']
    .rolling(15, min_periods=15)
    .mean()
    .shift(-14)  # damit die Summe bei t=07:02 die Werte von 07:02–07:16 enthält
    .reset_index(level=0, drop=True)
)


print(df_all.shape)
df_all = df_all[df_all['Datetime'].dt.time <= time(9, 45)]
print(df_all.shape)

#print(df_lagged_all["Datetime"]dt.time.min(), df_lagged_all["Datetime"].dt.date.max())
#df_all.head(20)
#print(df_lagged_all.isna().sum())


(76171, 15)
(73006, 15)


In [167]:
def create_sequences_sum(X, y, seq_length=15, horizon=15):
    Xs, ys = [], []
    for i in range(len(X) - seq_length - horizon + 1):
        Xs.append(X[i:i+seq_length])
        # statt array von 15 Werten → Summe
        ys.append(y[i+seq_length:i+seq_length+horizon].sum())  
    return np.array(Xs), np.array(ys)


In [204]:
early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)

            

In [205]:
split = int(len(X) * 0.8)
df_train, df_test = df_all[:split], df_all[split:]


X_train = df_train[flow_features_neighbour].values
y_train = df_train['FLOW_future_sum'].values
X_test = df_test[flow_features_neighbour].values
y_test = df_test['FLOW_future_sum'].values


X_train_seq, y_train_seq = create_sequences_sum(X_train, y_train, seq_length=15, horizon=15)
X_test_seq, y_test_seq = create_sequences_sum(X_test, y_test, seq_length=15, horizon=15)


X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train_seq.reshape(-1, X_train_seq.shape[2])).reshape(X_train_seq.shape)
X_test_scaled  = X_scaler.transform(X_test_seq.reshape(-1, X_test_seq.shape[2])).reshape(X_test_seq.shape)

y_train_scaled = y_scaler.fit_transform(y_train_seq.reshape(-1, 1))
y_test_scaled  = y_scaler.transform(y_test_seq.reshape(-1, 1))


n_features = X_train_scaled.shape[2]

checkpoint = ModelCheckpoint("model/LSTM_model_flow_neighbour_best.keras", monitor='val_rmse', save_best_only=True, mode='min')

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15, n_features), return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=True),
    LSTM(64, activation='tanh'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
#model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1,verbose=1)
hist = model.fit(
                X_train_scaled, y_train_scaled,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr, checkpoint],
                verbose=1
            )

y_pred = model.predict(X_test_scaled)

rmse = mean_squared_error(y_test_scaled, y_pred, squared=False)
mae = mean_absolute_error(y_test_scaled, y_pred)
r2 = r2_score(y_test_scaled, y_pred)

print(f"LSTM 1 output (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

Epoch 1/100


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


320/320 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.1518 - rmse: 0.3896 - val_loss: 0.2617 - val_rmse: 0.5116 - learning_rate: 0.0010
Epoch 2/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.1194 - rmse: 0.3455 - val_loss: 0.2489 - val_rmse: 0.4989 - learning_rate: 0.0010
Epoch 3/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.1129 - rmse: 0.3361 - val_loss: 0.2693 - val_rmse: 0.5190 - learning_rate: 0.0010
Epoch 4/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.1097 - rmse: 0.3313 - val_loss: 0.2553 - val_rmse: 0.5053 - learning_rate: 0.0010
Epoch 5/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.1082 - rmse: 0.3290 - val_loss: 0.2957 - val_rmse: 0.5438 - learning_rate: 0.0010
Epoch 6/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.1018 - rmse: 0.3191 - val_loss: 0.2295 - val_rmse: 0.4790 - learning_rate: 5.0000e-04
Epoch 7/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0995 - rmse: 0.3154 - val_loss: 0.2269 - val_rmse: 0.4764 - learning

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [206]:
df_test_peak= df_test.set_index("Datetime").between_time("07:15", "08:30").reset_index()
X_test_peak = df_test_peak[flow_features_neighbour].values
y_test_peak = df_test_peak['FLOW_future_sum'].values
X_test_seq_peak, y_test_seq_peak = create_sequences_sum(X_test_peak, y_test_peak, seq_length=15, horizon=15)

X_test_peak_scaled  = X_scaler.transform(X_test_seq_peak.reshape(-1, X_test_seq_peak.shape[2])).reshape(X_test_seq_peak.shape)
y_test_peak_scaled  = y_scaler.transform(y_test_seq_peak.reshape(-1, 1))

y_pred = model.predict(X_test_peak_scaled)

rmse = mean_squared_error(y_test_peak_scaled, y_pred, squared=False)
mae = mean_absolute_error(y_test_peak_scaled, y_pred)
r2 = r2_score(y_test_peak_scaled, y_pred)

print(f"LSTM 1 output (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
LSTM 1 output (FLOW) -> RMSE: 0.390, MAE: 0.266, R2: 0.727


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
